In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.4 MB/s eta 0:00:00




# Summary

1. **Explanation of the code**  
2. **Explanation of why the Pauli-Z is necessary in the Jordan-Wigner transformation**  
3. **Code section**  



## Explanation of the code

The molecule of $ H_2 $ is simply the "composition" of two hydrogen atoms. We are interested in finding the **ground state** of the $ H_2 $ molecule. To achieve this, we can use the **Variational Quantum Eigensolver (VQE)** algorithm to find this ground state.

The VQE is a hybrid quantum-classical algorithm designed to find the minimum energy (ground state) of quantum systems. It works by parameterizing a quantum circuit (ansatz) to prepare quantum states. The parameters of this circuit are iteratively adjusted by a classical optimizer to minimize the expected value of the system's energy, calculated from the Hamiltonian. In this way, the VQE combines quantum and classical resources to efficiently solve complex quantum mechanical problems.

In the context of $H_2 $, it is necessary to find the Hamiltonian of this molecule, which can be written as:

$
H = \sum_{p,q} h_{pq} a_p^\dagger a_q + \frac{1}{2} \sum_{p,q,r,s} h_{pqrs} a_p^\dagger a_q^\dagger a_r a_s
$

Where \( h_{pq} \) and \( h_{pqrs} \) are solutions to the following integrals:

$
h_{pq} = \int dx\, \phi_p^*(x) \left( -\frac{\nabla^2}{2} - \sum_{i=1}^N \frac{Z_i}{|x - R_i|} \right) \phi_q(x)
$

$
h_{pqrs} = \int dx_1 dx_2\, \phi_p^*(x_1) \phi_q^*(x_2) \frac{1}{|x_1 - x_2|} \phi_r(x_2) \phi_s(x_1)
$

**Note:** The Hamiltonian is given in terms of **fermionic operators** $ a_p^\dagger $ and $ a_q $, which describe the creation and annihilation of fermions (electrons). However, when working with quantum computing, these operators are incompatible with quantum logic gates. Therefore, it is necessary to **map** these fermionic operators into qubit operators that can be implemented using quantum gates.

There are several types of mappings, but the Jordan-Wigner transformation is recommended here. We can perform this mapping explicitly using `qml.qchem.Molecule()` from the PennyLane library, where we configure the parameters as:

```python
# Define fixed molecular parameters
coordinates = pnp.array([[x1, y1, z1], [x2, y2, z2]])
charge = 0
symbols = ['H', 'H']
multiplicity = 1
basis_set = "sto-3g"
electrons = 2
```

Then, we explicitly specify the Jordan-Wigner transformation in the function that generates the Hamiltonian:

```python
# Generate the molecular Hamiltonian and the necessary number of qubits
H, qubits = qchem.molecular_hamiltonian(
    molecule.symbols, molecule.coordinates, name="h2",
    charge=molecule.charge, mult=molecule.mult, basis=molecule.basis_name, mapping='jordan_wigner'
)
```

With the Hamiltonian in hand, we can proceed with the optimization procedure, which consists of:

1. **Finding a good ansatz:** This can be done using the method `qml.qchem.hf_state` to generate the Hartree-Fock state, which serves as a starting point for the optimization.

2. **Preparing a circuit with this state:** We define a quantum circuit that prepares the Hartree-Fock state and applies parameterized quantum gates to explore the state space.

3. **Calculating the expected value of the Hamiltonian:** We measure the expectation value of the Hamiltonian for the state prepared by the circuit, which depends on the parameters we are optimizing.

Here's how we implement this in code:

```python
# Initial Hartree-Fock state
hf = qml.qchem.hf_state(electrons, qubits)

# Define the quantum device
dev = qml.device("default.qubit", wires=qubits)

# Parametrized quantum circuit
@qml.qnode(dev, interface="jax")
def circuit_base(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.expval(H)
```

- `qml.BasisState(hf, wires=wires)`: Prepares the Hartree-Fock state on the qubits.
- `qml.DoubleExcitation(param, wires=[0, 1, 2, 3])`: Applies a parameterized double excitation gate, allowing us to capture electron correlation beyond the Hartree-Fock approximation.
- `qml.expval(H)`: Measures the expected value of the Hamiltonian \( H \) for the current state.

We define the cost function and set up the optimization:

```python
# Cost function for optimization
def cost_fn(param):
    return circuit_base(param, wires=range(qubits))

# Optimization parameters
max_iterations = 100
conv_tol = 1e-07
opt = optax.sgd(learning_rate=0.4)
theta = np.array(0.0)

# Store energy values and angles
energy = [cost_fn(theta)]
angle = [theta]
opt_state = opt.init(theta)

# Optimization loop
for n in range(max_iterations):
    gradient = jax.grad(cost_fn)(theta)
    updates, opt_state = opt.update(gradient, opt_state)
    theta = optax.apply_updates(theta, updates)

    angle.append(theta)
    energy.append(cost_fn(theta))

    conv = np.abs(energy[-1] - energy[-2])

    if conv <= conv_tol:
        break
```

- `jax.grad(cost_fn)(theta)`: Computes the gradient of the cost function with respect to the parameter `theta`.
- The optimizer adjusts `theta` to minimize the energy calculated by the circuit.
- The loop continues until the change in energy between iterations is less than the convergence tolerance `conv_tol`.



# Explanation of why the Pauli-Z is necessary in the Jordan-Wigner transformation

**It's because we need to preserve the anticommutation relation, which is very important for performing the correct simulation.**




## Code section

In [11]:
import pennylane as qml
from pennylane import qchem
import pennylane.numpy as pnp
from jax import numpy as np
import jax
import optax
import warnings

def VQE(x1, y1, z1, x2, y2, z2):
    """
    Perform the Variational Quantum Eigensolver (VQE) algorithm to find the
    ground state energy of a molecular system given the coordinates of its atoms.

    Args:
        x1, y1, z1: Coordinates of the first atom (floats).
        x2, y2, z2: Coordinates of the second atom (floats).

    Returns:
        float: The converged energy of the molecule in Hartree.
    """
    # Convert coordinates to a PennyLane-compatible NumPy array
    coordinates = pnp.array([[x1, y1, z1], [x2, y2, z2]])

    # Define fixed molecular parameters
    charge = 0
    symbols = ['H', 'H']
    multiplicity = 1
    basis_set = "sto-3g"
    electrons = 2

    # Create the molecule object
    molecule = qml.qchem.Molecule(
        symbols,
        coordinates,
        charge=charge,
        mult=multiplicity,
        basis_name=basis_set
    )

    # Generate the molecular Hamiltonian and the necessary number of qubits
    H, qubits = qchem.molecular_hamiltonian(
        molecule.symbols, molecule.coordinates, name="h2",
        charge=molecule.charge, mult=molecule.mult, basis=molecule.basis_name, mapping='jordan_wigner'
    )

    # Initial Hartree-Fock state
    hf = qml.qchem.hf_state(electrons, qubits)

    # Define the quantum device
    dev = qml.device("default.qubit", wires=qubits)

    # Parametrized quantum circuit
    @qml.qnode(dev, interface="jax")
    def circuit_base(param, wires):
        qml.BasisState(hf, wires=wires)
        qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
        return qml.expval(H)

    # Cost function for optimization
    def cost_fn(param):
        return circuit_base(param, wires=range(qubits))

    # Optimization parameters
    max_iterations = 100
    conv_tol = 1e-07
    opt = optax.sgd(learning_rate=0.4)
    theta = np.array(0.0)

    # Store energy values and angles
    energy = [cost_fn(theta)]
    angle = [theta]
    opt_state = opt.init(theta)

    # Optimization loop
    for n in range(max_iterations):
        gradient = jax.grad(cost_fn)(theta)
        updates, opt_state = opt.update(gradient, opt_state)
        theta = optax.apply_updates(theta, updates)

        angle.append(theta)
        energy.append(cost_fn(theta))

        conv = np.abs(energy[-1] - energy[-2])

        if conv <= conv_tol:
            break

    # Return the converged energy
    return energy[-1]

# Suppress warnings from PennyLane and JAX
warnings.filterwarnings("ignore", category=UserWarning, module="pennylane")
warnings.filterwarnings("ignore", category=UserWarning, module="jax")

# Suppress ComplexWarning warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)

# Define new coordinates for the hydrogen molecule
new_coordinates = (0, 0, -0.6614, 0, 0, 0.6614)
converged_energy = VQE(*new_coordinates)



Converged Energy: -1.13618922


In [12]:
print(f"Converged Energy: {converged_energy:.8f}")

Converged Energy: -1.13618922
